<a href="https://colab.research.google.com/github/MIT-LCP/2019_tokyo_datathon/blob/master/eicu_python/01_explore_patients_satoshi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# eICU Collaborative Research Database

# Notebook 1: Exploring the patient table：表の作り方

In this notebook we introduce the patient table, a key table in the [eICU Collaborative Research Database](http://eicu-crd.mit.edu/). The patient table contains patient demographics and admission and discharge details for hospital and ICU stays. For more detail, see: http://eicu-crd.mit.edu/eicutables/patient/

患者背景、入室、退室、ICU滞在期間などの表の作り方を説明します。

## Load libraries and connect to the data
Run the following cells to import some libraries and then connect to the database.

以下の操作により、必要なlibraryとデータベースへのアクセスが可能となります。

In [0]:
# Import libraries
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.path as path

# Make pandas dataframes prettier
from IPython.display import display, HTML

# Access data using Google BigQuery.
from google.colab import auth
from google.cloud import bigquery

As before, you need to first authenticate yourself by running the following cell. If you are running it for the first time, it will ask you to follow a link to log in using your Gmail account, and accept the data access requests to your profile. Once this is done, it will generate a string of verification code, which you should paste back to the cell below and press enter.

まず、下のセルを走らせて（`Run`をクリック）ください。提示されてリンクに従い、各自で"verification code"をコピー&ペーストしてください。各自のgmail accountからアクセスすることが可能になります。

In [0]:
auth.authenticate_user()

We'll also set the project details.

In [0]:
project_id='datathonjapan2019'
os.environ["GOOGLE_CLOUD_PROJECT"]=project_id

To make our lives easier, finally we'll install and import a set of helper functions from the `datathon2` package.

In [0]:
!pip install datathon2

In [0]:
import datathon2 as dtn

ここまでが準備段階です。以下、データの抽出とテーブル作成に移ります。

# Load data from the `patient` table

Now we can start exploring the data. We'll begin by running a simple query on the database to load all columns of the `patient` table to a Pandas DataFrame. The query is written in SQL, a common language for extracting data from databases. The structure of an SQL query is:

以下が、SQLを使ってデータを抽出（query）する基本的な構文です。`SELECT`で列を、`FROM`でテーブルを指定します。オプションで`WHERE`でより細かな条件を指定できます。

```sql
SELECT <columns>
FROM <table>
WHERE <criteria, optional>
```

`*` is a wildcard that indicates all columns

`*`を用いることで、全ての列を抽出できます。

In [0]:
query = """
SELECT *
FROM `physionet-data.eicu_crd_demo.patient`
"""

patient = dtn.run_query(query,project_id)

We have now assigned the output to our query to a variable called `patient`. Let's use the `head` method to view the first few rows of our data.

In [0]:
# view the top few rows of the patient data
patient.head()

## Questions

- What does `patientunitstayid` represent? (hint, see: http://eicu-crd.mit.edu/eicutables/patient/)　

（`patientunitstayid`列の意味は？）

- What does `patienthealthsystemstayid` represent?

（`patienthealthsystemstayid`列の意味は？）

- What does `uniquepid` represent?

（`uniquepid`列の意味は？）

In [0]:
# select a limited number of columns to view
columns = ['uniquepid', 'patientunitstayid','gender','age','unitdischargestatus']
patient[columns].head()

- Try running the following query, which lists unique values in the age column. What do you notice?

- `age`の分布をヒストグラムで描いてみましょう。予想通りですか？

In [0]:
# what are the unique values for age?
age_col = 'age'
patient[age_col].sort_values().unique()

- Try plotting a histogram of ages using the command in the cell below. What happens? Why?

In [0]:
# try plotting a histogram of ages
patient[age_col].plot(kind='hist', bins=15)

Let's create a new column named `age_num`, then try again.

In [0]:
# create a column containing numerical ages
# If ‘coerce’, then invalid parsing will be set as NaN
agenum_col = 'age_num'
patient[agenum_col] = pd.to_numeric(patient[age_col], errors='coerce')
patient[agenum_col].sort_values().unique()

In [0]:
patient[agenum_col].plot(kind='hist', bins=15)

## Questions

- Use the `mean()` method to find the average age. Why do we expect this to be lower than the true mean?

（`mean`を用いて年齢の平均を出し、本当の平均値と比較してみましょう。）
- In the same way that you use `mean()`, you can use `describe()`, `max()`, and `min()`. Look at the admission heights (`admissionheight`) of patients in cm. What issue do you see? How can you deal with this issue?

（同様に`describe()`, `max()`, `min()`を使ってみて下さい。どこか問題がありませんか？）

In [0]:
adheight_col = 'admissionheight'
patient[adheight_col].describe()

In [0]:
# set threshold
adheight_col = 'admissionheight'
patient[patient[adheight_col] < 10] = None